## Imports

In [1]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, DelimitedFiles, Combinatorics

## Read in data, initialize parameters

In [2]:
# number of crews
C = 10

# get inital fire perimeters and no-suppression progression parameters
M = readdlm("data/processed/sample_growth_patterns.csv", ',')
start_perims = M[:, 1]
progressions = M[:, 2:15]

# time periods
T = size(M)[2] - 1

# number of fires
G = size(M)[1]

# get distance from fire f to fire g (dimension G-by-G)
fire_dists =  readdlm("data/processed/fire_distances.csv", ',')

# get distance from base c to fire g (dimension C-by-G)
base_fire_dists =  readdlm("data/processed/base_fire_distances.csv", ',')

# initialize travel times (number of periods) from fire f to fire g (dimension G-by-G)
tau = convert(Array{Int}, ones(size(fire_dists)))

# initialize travel times (number of periods) from base c to fire g (dimension C-by-G)
tau_base_to_fire = convert(Array{Int}, ones((size(base_fire_dists))))

# read intial crew statuses (location, period by which they must rest)
# (-1 in current_fire means crew is currently at base)
# (rested_periods is the amount of time crew has been at base, relevant for completing rest)
crew_starts = CSV.read("data/processed/sample_crew_starts.csv", DataFrame)
rest_by = crew_starts[!, "rest_by"]
current_fire = crew_starts[!, "current_fire"]
rested_periods = crew_starts[!, "rested_periods"]

# how long at base to be considered "rested"
break_length = 2

## some parameters that make tradeoffs reasonable ##

# cost of one area unit burned / cost of mile traveled
beta = 100

# cost of crew-day of suppression / cost of mile traveled
alpha = 200

# how much perimeter prevented per crew per time period
line_per_crew = 17

display((C, T, G))
display(base_fire_dists[1:C, 1:G])
display(crew_starts)

(10, 14, 3)

10×3 Matrix{Float64}:
 109.406  177.41    442.573
 512.873  620.197   677.291
 301.865  415.648   525.184
 202.729  285.549   548.429
 198.903   76.6787  275.938
 366.87   380.749   262.297
 343.768  358.437   630.655
 155.524   30.2902  262.352
 321.604  379.411   357.643
 110.707   85.7744  360.787

,crew,rest_by,current_fire,rested_periods
,Int64,Int64,Int64,Int64
1,1,15,-1,-1
2,2,15,-1,-1
3,3,10,1,-1
4,4,10,1,-1
5,5,8,4,-1
6,6,8,-1,0
7,7,5,4,-1
8,8,5,4,-1
9,9,4,-1,1


## Define arc sets (most slippery part, should write a unit test)

In [3]:
## generate fire_to_fire index set

# arcs for time-space network for non-rested crews
non_rested_ff = [(c, f_from, f_to, t_from, t_from + tau[f_to, f_from], 0)
                  for c=1:C, f_from=1:G, f_to=1:G, t_from=1:T
                  if t_from <= rest_by[c]]

# arcs for time-space network for rested crews
rested_ff = [(c, f_from, f_to, t_from, t_from + tau[f_to, f_from], 1)
               for c=1:C, f_from=1:G, f_to=1:G, t_from=1:T
               if 1==1]

# special arcs for first travel from initial fires
from_start_ff = [(c, current_fire[c], f_to, 0, tau[f_to, current_fire[c]], 0)
                  for c=1:C, f_to=1:G
                  if current_fire[c] != -1]

# concat
ff_ix = vcat(non_rested_ff, rested_ff)
ff_ix = vcat(from_start_ff, ff_ix)

## generate fire_to_rest index set

# arcs for time-space network for non-rested crews
non_rested_fr = [(c, f_from, t_from, t_from + tau_base_to_fire[c, f_from], 0)
                  for c=1:C, f_from=1:G, t_from=1:T
                  if t_from <= rest_by[c]]

# arcs for time-space network for rested crews
rested_fr = [(c, f_from, t_from, t_from + tau_base_to_fire[c, f_from], 1)
               for c=1:C, f_from=1:G, t_from=1:T
               if 1==1]

# special arcs for first travel from initial fires
from_start_fr = [(c, current_fire[c], 0, tau_base_to_fire[c, current_fire[c]], 0)
                  for c=1:C
                  if current_fire[c] != -1]

# concat
fr_ix = vcat(non_rested_fr, rested_fr)
fr_ix = vcat(from_start_fr, fr_ix)

## generate rest_to_fire index set

# arcs for time-space network when leaving base without rest
non_rested_rf = [(c, f_to, t_from, t_from + tau_base_to_fire[c, f_to], 0)
                   for c=1:C, f_to=1:G, t_from=1:T
                   if t_from <= rest_by[c]]

# arcs for time-space network when going non-rest -> rest
rested_rf = [(c, f_to, t_from, t_from + tau_base_to_fire[c, f_to], 1)
               for c=1:C, f_to=1:G, t_from=1:T
               if 1==1]

# special arcs for first travel from initial rest
from_start_rf = [(c, f_to, 0, tau_base_to_fire[c, f_to], 0)
                  for c=1:C, f_to=1:G
                  if current_fire[c] == -1]

# concat
rf_ix = vcat(non_rested_rf, rested_rf)
rf_ix = vcat(from_start_rf, rf_ix)

## generate rest_to_rest index set
## (crew, time_from, time_to, rested_From, rested_to)


non_to_non_rested_rr = [(c, t_from, t_from + 1, 0, 0)
                           for c=1:C, t_from=1:T
                           if t_from <= rest_by[c]]

non_to_yes_rested_rr = [(c, t_from, t_from + break_length, 0, 1)
                           for c=1:C, t_from=1:T
                           if t_from <= rest_by[c]]

yes_rested_rr = [(c, t_from, t_from + 1, 1, 1)
                   for c=1:C, t_from=1:T
                    if 1==1]

from_start_to_non_rested_rr = [(c, 0, 1, 0, 0) for c=1:C if current_fire[c] == -1]
from_start_to_yes_rested_rr = [(c, 0, break_length - rested_periods[c], 0, 1) 
                                  for c=1:C 
                                  if (current_fire[c] == -1) & (rested_periods[c] != -1)]


rr_ix = vcat(non_to_non_rested_rr, non_to_yes_rested_rr)
rr_ix = vcat(rr_ix, yes_rested_rr)
rr_ix = vcat(from_start_to_non_rested_rr, rr_ix)
rr_ix = vcat(from_start_to_yes_rested_rr, rr_ix)

ff_ix_arr = [[ix for ix in ff_ix if ix[1] == c] for c=1:C]
fr_ix_arr = [[ix for ix in fr_ix if ix[1] == c] for c=1:C]
rf_ix_arr = [[ix for ix in rf_ix if ix[1] == c] for c=1:C]
rr_ix_arr = [[ix for ix in rr_ix if ix[1] == c] for c=1:C]

size(ff_ix), size(fr_ix), size(rf_ix), size(rr_ix)

((1989,), (663,), (669,), (304,))

## Make model

In [4]:
m = Model(Gurobi.Optimizer)

# fire suppression plan section
@variable(m, p[g=1:G, t=1:T+1] >= 0)
@variable(m, l[g=1:G, t=1:T] >= 0)
@constraint(m, perim_growth[g=1:G, t=1:T], p[g, t+1] >= progressions[g, t] * (p[g, t] - l[g, t] / 2)                                                             - l[g, t] / 2)
@constraint(m, perim_start[g=1:G], p[g, 1] == start_perims[g])

# routing plan section
@variable(m, ff[ff_ix] >= 0)
@variable(m, fr[fr_ix] >= 0)
@variable(m, rf[rf_ix] >= 0)
@variable(m, rr[rr_ix] >= 0)


@constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=0:1],
    
            # outflow
            sum(ff[key] for key in ff_ix_arr[c]
                    if (key[2] == g) & (key[4] == t) & (key[6] == rest)
                ) +    
    
            sum(fr[key] for key in fr_ix_arr[c]
                        if (key[2] == g) & (key[3] == t) & (key[5] == rest)
                ) 
    
    ==
            # inflow
            sum(ff[key] for key in ff_ix_arr[c]
                        if (key[3] == g) & (key[5] == t) & (key[6] == rest)
                ) +
    
            sum(rf[key] for key in rf_ix_arr[c]
                        if (key[2] == g) & (key[4] == t) & (key[5] == rest)
                ) 
    
            )   

@constraint(m, rest_flow[c=1:C, t=1:T, rest=0:1], 
    
            # outflow
            sum(rf[key] for key in rf_ix_arr[c]
                        if (key[3] == t) & (key[5] == rest)
                ) +
    
            sum(rr[key] for key in rr_ix_arr[c]
                        if (key[2] == t) & (key[4] == rest)
                )
    
            ==       
    
            # inflow
            sum(fr[key] for key in fr_ix_arr[c]
                        if (key[4] == t) &  (key[5] == rest)
                ) +
            sum(rr[key] for key in rr_ix_arr[c]
                        if (key[3] == t) & (key[5] == rest)
                )
           )

@constraint(m, start[c=1:C], 
    
    sum(ff[key] for key in from_start_ff if key[1] == c) + 
    sum(rf[key] for key in from_start_rf if key[1] == c) + 
    sum(fr[key] for key in from_start_fr if key[1] == c) + 
    sum(rr[key] for key in from_start_to_non_rested_rr if key[1] == c) + 
    sum(rr[key] for key in from_start_to_yes_rested_rr if key[1] == c)== 1
           )


# linking constraints
@constraint(m, linking[g=1:G, t=1:T],
    
     sum(ff[key] for key in ff_ix if (key[3] == g) & (key[5] == t)) + 
     sum(rf[key] for key in rf_ix if (key[2] == g) & (key[4] == t))
        >= l[g, t] / line_per_crew
    
           );

Academic license - for non-commercial use only


## Run model

In [5]:
@objective(m, Min, 
    beta * (sum(p) - sum(p[1:G, 1])/2 - sum(p[1:G, T+1])/2) + 
    sum(ff[key] * (alpha + fire_dists[key[2], key[3]]) for key in ff_ix) +
    sum(fr[key] * (base_fire_dists[key[1], key[2]]) for key in fr_ix) + 
    sum(rf[key] * (alpha + base_fire_dists[key[1], key[2]]) for key in rf_ix)

)

optimize!(m);

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1217 rows, 3712 columns and 9739 nonzeros
Model fingerprint: 0x1d643ff0
Coefficient statistics:
  Matrix range     [6e-02, 4e+00]
  Objective range  [3e+01, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 566 rows and 996 columns
Presolve time: 0.01s
Presolved: 651 rows, 2716 columns, 7469 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6295548e+05   3.567755e+02   0.000000e+00      0s
    1136    1.1445329e+06   0.000000e+00   0.000000e+00      0s

Solved in 1136 iterations and 0.03 seconds
Optimal objective  1.144532856e+06

User-callback calls 1184, time in user-callback 0.01 sec


In [6]:
# show the perimeter progressions and number of crews suppressing
show(stdout, "text/plain", value.(p))
print("\n\n")
show(stdout, "text/plain", value.(l)/line_per_crew)

3×15 Matrix{Float64}:
 1000.0  1500.0  1500.0  1500.0  1350.0  1167.5  937.7  627.76  379.808  181.446  40.3125  0.0  0.0  0.0  0.0
  500.0   500.0   375.0   222.0    69.0     0.0    0.0    0.0     0.0      0.0     0.0     0.0  0.0  0.0  0.0
  100.0    17.5     0.0     0.0     0.0     0.0    0.0    0.0     0.0      0.0     0.0     0.0  0.0  0.0  0.0

3×14 Matrix{Float64}:
 0.0  0.0      0.0  0.0  2.94118  7.0  8.0  8.0  8.0  6.0  1.95285  0.0  0.0  0.0
 0.0  7.35294  9.0  9.0  4.05882  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0
 9.0  1.64706  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0

## Generate starting columns for CG approach

### Fire suppression plans

In [7]:
function fire_plan_cost(start_perim, line_per_day, progression)
    
    perim = start_perim
    area = 0
    new_perim = 0
    for i in 1:length(progression)
        new_perim = (perim - line_per_day[i]/2) * progression[i] - line_per_day[i]/2
        new_perim = max(new_perim, 0)
        area = area + perim/2 + new_perim/2
        perim = new_perim
    end
    area
end

fire_plan_cost (generic function with 1 method)

In [8]:
plans_per_fire = 20
B = zeros((plans_per_fire, G, T))
fire_costs = zeros((plans_per_fire, G));

First we round down the crews per day to integers, and then we take away as few crews as possible while enumerating the desired number of plans.

For example [2, 2, 2, 2, 1, 1, 0] will give plan options like
- [2, 2, 2, 2, 1, 1, 0] 
- [1, 2, 2, 2, 1, 1, 0] 
- [2, 1, 2, 2, 1, 1, 0] 
- [2, 2, 1, 2, 1, 1, 0] 

etc.

In [9]:
function perturb(list, round_ix, first, last, amount)
    [i in round_ix[first+1:last] ? list[i] + amount : list[i] for i in 1:length(list)]
end
    

perturb (generic function with 1 method)

In [10]:
# hpw many plans to make per fire
plans_per_fire = 20

# initialize list of number of perturbations found for each fire (current process might not yield
# enough plans)
found_plans = convert.(Int, zeros(G))

# initialize costs and allocations
B = zeros((plans_per_fire, G, T))
fire_costs = zeros((plans_per_fire, G))

# for each fire
for fire in 1:G
    
    assignments = (value.(l)/line_per_crew)[fire, 1:T]
    assignments = round.(assignments)

    plan = 0
    perturb_amount = 1
    ix_to_round = [i for i in 1:T if (assignments[i] > perturb_amount - 0.99) & 
                                     (assignments[i] + perturb_amount < C + 0.01)]

    while (plan < plans_per_fire) & (length(ix_to_round) > 0)

        ixs = [(i, j) for i in 0:length(ix_to_round) for j in 0:length(ix_to_round) if i < j]

        for ix in ixs
            perturbed_up = perturb(assignments, ix_to_round, ix[1], ix[2], perturb_amount)
            perturbed_down = perturb(assignments, ix_to_round, ix[1], ix[2], -perturb_amount)

            plan = plan + 1
            if plan <= plans_per_fire
                found_plans[fire] = plan
                B[plan, fire, :] = perturbed_up

                fire_costs[plan, fire] = beta * fire_plan_cost(start_perims[fire], 
                                                   perturbed_up * line_per_crew, 
                                                   progressions[fire, :])
            end

            plan = plan + 1
            if plan <= plans_per_fire
                found_plans[fire] = plan
                B[plan, fire, :] = perturbed_down

                fire_costs[plan, fire] = beta * fire_plan_cost(start_perims[fire], 
                                                   perturbed_down * line_per_crew, 
                                                   progressions[fire, :])
            end
        end
        perturb_amount = perturb_amount + 1
        ix_to_round = [i for i in 1:T if (assignments[i] > perturb_amount - 0.99) & 
                                 (assignments[i] + perturb_amount < C + 0.01)]
    end
    
end

In [11]:
B[8, 1, :]

14-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 2.0
 6.0
 7.0
 7.0
 8.0
 6.0
 2.0
 0.0
 0.0
 0.0

In [38]:
function routes_from_plan(plans_arr, supp_plans)
    
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)


    # routing plan section
    @variable(m, ff[ff_ix], Bin)
    @variable(m, fr[fr_ix], Bin)
    @variable(m, rf[rf_ix], Bin)
    @variable(m, rr[rr_ix], Bin)


    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=0:1],

                # outflow
                sum(ff[key] for key in ff_ix_arr[c]
                        if (key[2] == g) & (key[4] == t) & (key[6] == rest)
                    ) +    

                sum(fr[key] for key in fr_ix_arr[c]
                            if (key[2] == g) & (key[3] == t) & (key[5] == rest)
                    ) 

        ==
                # inflow
                sum(ff[key] for key in ff_ix_arr[c]
                            if (key[3] == g) & (key[5] == t) & (key[6] == rest)
                    ) +

                sum(rf[key] for key in rf_ix_arr[c]
                            if (key[2] == g) & (key[4] == t) & (key[5] == rest)
                    ) 

                )   

    @constraint(m, rest_flow[c=1:C, t=1:T, rest=0:1], 

                # outflow
                sum(rf[key] for key in rf_ix_arr[c]
                            if (key[3] == t) & (key[5] == rest)
                    ) +

                sum(rr[key] for key in rr_ix_arr[c]
                            if (key[2] == t) & (key[4] == rest)
                    )

                ==       

                # inflow
                sum(fr[key] for key in fr_ix_arr[c]
                            if (key[4] == t) &  (key[5] == rest)
                    ) +
                sum(rr[key] for key in rr_ix_arr[c]
                            if (key[3] == t) & (key[5] == rest)
                    )
               )

    @constraint(m, start[c=1:C], 

        sum(ff[key] for key in from_start_ff if key[1] == c) + 
        sum(rf[key] for key in from_start_rf if key[1] == c) + 
        sum(fr[key] for key in from_start_fr if key[1] == c) + 
        sum(rr[key] for key in from_start_to_non_rested_rr if key[1] == c) + 
        sum(rr[key] for key in from_start_to_yes_rested_rr if key[1] == c)== 1
               )


    # linking constraints
    @constraint(m, linking[g=1:G, t=1:T],

         sum(ff[key] for key in ff_ix if (key[3] == g) & (key[5] == t)) + 
         sum(rf[key] for key in rf_ix if (key[2] == g) & (key[4] == t))
            >= supp_plans[plans_arr[g], g, t]

               );

    @objective(m, Min, 
        sum(ff[key] * (alpha + fire_dists[key[2], key[3]]) for key in ff_ix) +
        sum(fr[key] * (base_fire_dists[key[1], key[2]]) for key in fr_ix) + 
        sum(rf[key] * (alpha + base_fire_dists[key[1], key[2]]) for key in rf_ix)
    )

    optimize!(m)
    
    opt = false
    fires_fought = [(0)]
    route_costs = [1]
    if termination_status(m) == MOI.OPTIMAL
        opt = true
        fires_fought = vcat([(ix[1], ix[3], ix[5]) for ix in ff_ix if (value(ff[ix]) > 0.99)],
                        [(ix[1], ix[2], ix[4]) for ix in rf_ix if (value(rf[ix]) > 0.99)])
        
        route_costs = 
        [sum(value(ff[key]) * (alpha + fire_dists[key[2], key[3]]) for key in ff_ix_arr[crew]) +
        sum(value(fr[key]) * (base_fire_dists[key[1], key[2]]) for key in fr_ix_arr[crew]) + 
        sum(value(rf[key]) * (alpha + base_fire_dists[key[1], key[2]]) for key in rf_ix_arr[crew])
            for crew = 1:C]
    end
    opt, fires_fought, route_costs
end

routes_from_plan (generic function with 2 methods)

In [39]:
trials = 10
samples = unique([[rand(1:found_plans[g]) for g in 1:G] for i in 1:20*trials])
samples = [i for i in samples if sum(mod.(i, 2)) == 0][1:trials]

10-element Vector{Vector{Int64}}:
 [8, 14, 2]
 [8, 12, 2]
 [20, 16, 6]
 [16, 8, 6]
 [16, 4, 8]
 [2, 20, 8]
 [2, 18, 4]
 [18, 14, 8]
 [2, 6, 4]
 [8, 18, 2]

In [49]:
dummy = routes_from_plan([1, 1, 1], zeros(size(B)))

Academic license - for non-commercial use only


(true, Tuple{Int64, Int64, Int64}[], [0.0, 0.0, 301.86514967206256, 202.72917060159685, 248.55189457564524, 0.0, 388.0961083852714, 201.87621790011204, 0.0, 168.80604685654075])

In [40]:
all_routes = [routes_from_plan(sample, B) for sample in samples];

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [44]:
all_routes

10-element Vector{Tuple{Bool, Vector{Tuple{Int64, Int64, Int64}}, Vector{Float64}}}:
 (1, [(3, 3, 1), (4, 3, 1), (5, 3, 1), (7, 3, 1), (8, 3, 1), (2, 2, 2), (3, 2, 2), (4, 2, 2), (5, 2, 2), (7, 2, 2)  …  (10, 1, 10), (9, 1, 11), (1, 3, 1), (2, 3, 1), (6, 3, 1), (9, 2, 2), (10, 2, 4), (10, 1, 6), (5, 1, 7), (8, 1, 9)], [3152.2808261540376, 3590.465890663369, 2853.4675421846036, 2754.3315631141377, 2508.578576377684, 924.5942180600978, 1792.5317583419405, 2175.4325442808777, 2826.2481047524516, 1761.7693102953313])
 (1, [(3, 3, 1), (4, 3, 1), (5, 3, 1), (7, 3, 1), (8, 3, 1), (1, 2, 2), (2, 2, 2), (3, 2, 2), (5, 2, 2), (7, 2, 2)  …  (9, 1, 10), (10, 1, 10), (5, 1, 11), (1, 3, 1), (2, 3, 1), (6, 3, 1), (9, 2, 2), (10, 2, 4), (5, 1, 7), (8, 1, 9)], [3152.2808261540376, 3590.465890663369, 2853.4675421846036, 2754.3315631141377, 2708.578576377684, 924.5942180600978, 1792.5317583419405, 2175.4325442808777, 2626.2481047524516, 1890.521390237407])
 (1, [(3, 3, 1), (4, 3, 1), (5, 3, 1), (7, 3, 1)

In [56]:
all_routes = vcat(all_routes, dummy)

11-element Vector{Tuple{Bool, Vector{Tuple{Int64, Int64, Int64}}, Vector{Float64}}}:
 (1, [(3, 3, 1), (4, 3, 1), (5, 3, 1), (7, 3, 1), (8, 3, 1), (2, 2, 2), (3, 2, 2), (4, 2, 2), (5, 2, 2), (7, 2, 2)  …  (10, 1, 10), (9, 1, 11), (1, 3, 1), (2, 3, 1), (6, 3, 1), (9, 2, 2), (10, 2, 4), (10, 1, 6), (5, 1, 7), (8, 1, 9)], [3152.2808261540376, 3590.465890663369, 2853.4675421846036, 2754.3315631141377, 2508.578576377684, 924.5942180600978, 1792.5317583419405, 2175.4325442808777, 2826.2481047524516, 1761.7693102953313])
 (1, [(3, 3, 1), (4, 3, 1), (5, 3, 1), (7, 3, 1), (8, 3, 1), (1, 2, 2), (2, 2, 2), (3, 2, 2), (5, 2, 2), (7, 2, 2)  …  (9, 1, 10), (10, 1, 10), (5, 1, 11), (1, 3, 1), (2, 3, 1), (6, 3, 1), (9, 2, 2), (10, 2, 4), (5, 1, 7), (8, 1, 9)], [3152.2808261540376, 3590.465890663369, 2853.4675421846036, 2754.3315631141377, 2708.578576377684, 924.5942180600978, 1792.5317583419405, 2175.4325442808777, 2626.2481047524516, 1890.521390237407])
 (1, [(3, 3, 1), (4, 3, 1), (5, 3, 1), (7, 3, 1)

In [72]:
unique_routes = [unique(x->x[1], [([i for i in route[2] if i[1] == c], route[3][c]) 
                                                for route in all_routes if route[1] == 1])
                        for c=1:C]

num_routes = [length(i) for i in unique_routes]
A = zeros((maximum(num_routes), C, G, T))
route_costs = zeros((maximum(num_routes), C))

for c in 1:C
    i = 0
    for route in unique_routes[c]
        i += 1
        assignments = route[1]
        cost = route[2]
        
        for assignment in assignments
            A[i, assignment[1], assignment[2], assignment[3]] = 1
        end
        route_costs[i, c] = cost
    end
end

In [77]:
A[1, 4, :, :]

3×14 Matrix{Float64}:
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [73]:
route_costs

10×10 Matrix{Float64}:
 3152.28  3590.47  2853.47   2754.33   …  2175.43   2826.25  1761.77
 3152.28  3590.47  2853.47   2754.33      2193.01   2626.25  1890.52
 2952.28  3790.47   301.865  2754.33      2193.01   3255.33  1590.22
 2952.28  3790.47     0.0     202.729     2575.43   2558.49  1890.52
 2952.28  3790.47     0.0       0.0       2375.43   2426.25  1961.77
 2913.86     0.0      0.0       0.0    …   201.876  2558.49  1390.22
    0.0      0.0      0.0       0.0          0.0    2426.25  2090.52
    0.0      0.0      0.0       0.0          0.0    2626.25  2090.52
    0.0      0.0      0.0       0.0          0.0       0.0    168.806
    0.0      0.0      0.0       0.0          0.0       0.0      0.0

In [222]:
fire_costs

20×3 Matrix{Float64}:
      9.85567e5  270900.0     1.16266e7
      9.84576e5  269200.0     4.12367e6
      9.83815e5  267500.0  6750.0
      9.83274e5  265800.0  5000.0
      9.82598e5  138300.0  5000.0
      9.82024e5  136600.0  5000.0
      9.81544e5  134900.0  5000.0
      9.57195e5  133200.0  5000.0
      9.56057e5  129100.0  5000.0
      9.55072e5  127400.0  5000.0
      9.54264e5  125700.0  5000.0
      9.53254e5  125700.0  5000.0
      9.52098e5  124000.0  5000.0
      9.51289e5  122300.0  5000.0
      9.38505e5  120600.0  5000.0
      9.37366e5  120600.0  5000.0
      9.36332e5  118900.0  5000.0
 935353.0        117200.0  5000.0
 934129.0        115500.0  5000.0
 932599.0        115500.0  5000.0

### Routing plans

To start we have two plans per crew, (1) that mimics the original flow to some extent and (2) the dummy all-rest flow.

(1) Round all flow values to nearest integer, make these the new arc costs, re-solve metwork flow problem as a "max-cost path"

(2) Solve same network flow minimizing fire fighting.

In [122]:
fire_plan_ix = [(i, g) for g=1:G for i=1:found_plans[g]]
route_plan_ix = [(i, c) for c=1:C for i=1:num_routes[c]]

cg = Model(Gurobi.Optimizer)

@variable(cg, suppression_plans[fire_plan_ix] >= 0)
@variable(cg, crew_routes[route_plan_ix] >= 0)

@constraint(cg, plan_per_fire[g=1:G], 
                sum(suppression_plans[ix] for ix in fire_plan_ix if ix[2] == g) >= 1)
@constraint(cg, route_per_crew[c=1:C], 
                sum(crew_routes[ix] for ix in route_plan_ix if ix[2] == c) == 1)
@constraint(cg, cover_plans[g=1:G, t=1:T],
                sum(crew_routes[ix] * A[ix[1], ix[2], g, t] for ix in route_plan_ix) >=
                sum(suppression_plans[ix] * B[ix[1], g, t] for ix in fire_plan_ix if ix[2] == g)
           )

@objective(cg, Min, 
              sum(crew_routes[ix] * route_costs[ix[1], ix[2]] for ix in route_plan_ix) + 
              sum(suppression_plans[ix] * fire_costs[ix[1], ix[2]] for ix in fire_plan_ix)
         )

optimize!(cg)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 55 rows, 112 columns and 761 nonzeros
Model fingerprint: 0xc5802655
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+02, 1e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 29 rows and 4 columns
Presolve time: 0.00s
Presolved: 26 rows, 108 columns, 757 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5119246e+03   3.000000e+00   0.000000e+00      0s
      62    1.2898946e+06   0.000000e+00   0.000000e+00      0s

Solved in 62 iterations and 0.00 seconds
Optimal objective  1.289894566e+06

User-callback calls 90, time in user-callback 0.00 sec


In [129]:
pi = dual.(plan_per_fire)
sigma = dual.(route_per_crew)
rho = dual.(cover_plans)

pi

3-element Vector{Float64}:
 1.1132575653759995e6
 1.2042428380239983e6
 1.4742265220160005e6

In [116]:
[ix for ix in fire_plan_ix if value(suppression_plans[ix]) > 0.01]

3-element Vector{Tuple{Int64, Int64}}:
 (20, 1)
 (2, 2)
 (5, 3)

In [119]:
fire_costs[2, 2]

270900.0

In [100]:
@variable(cg, suppression_plans[(21, 1)] >= 0)

LoadError: An object of name suppression_plans is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :suppression_plans)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:suppression_plans]`.


In [105]:
push!(fire_plan_ix, (21, 1))
fire_plan_ix

49-element Vector{Tuple{Int64, Int64}}:
 (1, 1)
 (2, 1)
 (3, 1)
 (4, 1)
 (5, 1)
 (6, 1)
 (7, 1)
 (8, 1)
 (9, 1)
 (10, 1)
 (11, 1)
 (12, 1)
 (13, 1)
 ⋮
 (18, 2)
 (19, 2)
 (20, 2)
 (1, 3)
 (2, 3)
 (3, 3)
 (4, 3)
 (5, 3)
 (6, 3)
 (7, 3)
 (8, 3)
 (21, 1)

In [94]:
suppression_plans[(21, 1)]

LoadError: KeyError: key (21, 1) not found

In [89]:
unique(value.(suppression_plans)), unique(value.(crew_routes))

([0.0, 1.0], [0.0, 0.6666666666666666, 0.33333333333333337, 0.6666666666666667, 0.3333333333333333, 1.0])

In [301]:
shadow_price.(cover_plans)

3×14 Matrix{Float64}:
 -0.0      -0.0      -0.0      -0.0  …  -287.614  -0.0  -0.0  -0.0  -0.0
 -0.0  -66300.0  -64600.0  -62900.0       -0.0    -0.0  -0.0  -0.0  -0.0
 -0.0      -0.0      -0.0      -0.0       -0.0    -0.0  -0.0  -0.0  -0.0

In [230]:
sum(value.(suppression_plans))

3.0

In [186]:
[ix for ix in fire_plan_ix if value(suppression_plans[ix]) > 0.99]

3-element Vector{Tuple{Int64, Int64}}:
 (7, 1)
 (4, 2)
 (2, 3)

In [181]:
unique(value.(suppression_plans))

2-element Vector{Float64}:
 0.0
 1.0

In [182]:
fire_costs

20×3 Matrix{Float64}:
 9.85567e5  270900.0       1.16266e7
 9.84576e5  269200.0       4.12367e6
 9.83815e5  267500.0       2.41286e7
 9.83274e5  265800.0       3.41311e7
 9.82598e5  627900.0       4.41336e7
 9.82024e5  626200.0       5.41361e7
 9.81544e5  624500.0       6.41385e7
 1.02066e6  622800.0       7.4141e7
 1.01967e6  984900.0       8.41435e7
 1.01891e6  983200.0       9.4146e7
 1.01837e6  981500.0       0.0
 1.01769e6  979800.0       0.0
 1.01712e6       1.3419e6  0.0
 1.05156e6       1.3402e6  0.0
 1.05079e6       1.3385e6  0.0
 1.05025e6       1.3368e6  0.0
 1.04958e6       1.6989e6  0.0
 1.049e6         1.6972e6  0.0
 1.0762e6        1.6955e6  0.0
 1.07544e6       1.9692e6  0.0

In [183]:
route_costs

2×10 Matrix{Float64}:
 2952.28  3790.47  2853.47   2754.33   …  2375.43   3055.33  1890.52
    0.0      0.0    301.865   202.729      201.876     0.0    168.806

In [184]:
cover_plans[3, 1]

cover_plans[3,1] : -8 suppression_plans[(1, 3)] - 9 suppression_plans[(2, 3)] - 7 suppression_plans[(3, 3)] - 6 suppression_plans[(4, 3)] - 5 suppression_plans[(5, 3)] - 4 suppression_plans[(6, 3)] - 3 suppression_plans[(7, 3)] - 2 suppression_plans[(8, 3)] - suppression_plans[(9, 3)] + crew_routes[(1, 1)] + crew_routes[(1, 2)] + crew_routes[(1, 3)] + crew_routes[(1, 4)] + crew_routes[(1, 5)] + crew_routes[(1, 6)] + crew_routes[(1, 7)] + crew_routes[(1, 8)] + crew_routes[(1, 9)] >= 0.0

In [185]:
B

20×3×14 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  8.0
 0.0  0.0  9.0
 0.0  0.0  7.0
 0.0  0.0  6.0
 0.0  0.0  5.0
 0.0  0.0  4.0
 0.0  0.0  3.0
 0.0  0.0  2.0
 0.0  0.0  1.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  6.0  1.0
 0.0  7.0  0.0
 0.0  7.0  0.0
 0.0  7.0  0.0
 0.0  5.0  0.0
 0.0  6.0  0.0
 0.0  6.0  0.0
 0.0  6.0  0.0
 0.0  4.0  0.0
 0.0  5.0  0.0
 0.0  5.0  0.0
 0.0  5.0  0.0
 0.0  3.0  0.0
 0.0  4.0  0.0
 0.0  4.0  0.0
 0.0  4.0  0.0
 0.0  2.0  0.0
 0.0  3.0  0.0
 0.0  3.0  0.0
 0.0  1.0  0.0

[:, :, 3] =
 0.0  9.0  0.0
 0.0  8.0  0.0
 0.0  9.0  0.0
 0.0  9.0  0.0
 0.0  8.0  0.0
 0.0  7.0  0.0
 0.0  8.0  0.0
 0.0  8.0  0.0
 0.0  7.0  0.0
 0.0  6.0  0.0
 0.0  7.0  0.0
 0.0  7.0  0.0
 0.0  6.0  0.0
 0.0  5.0  0.0
 0.0  6.0  0.0
 0.0  6.0  0.0
 0.0  5.0  0.0
 0.0  4.0  0.0
 0.0  5.0  0.0
 0.0  4.0  0.0

;;; … 

[:, :, 12] =
 0.0  0.0  0.

In [ ]:
function routes_from_plan(pi, sigma, rho)
    
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)


    # routing plan section
    @variable(m, ff[ff_ix], Bin)
    @variable(m, fr[fr_ix], Bin)
    @variable(m, rf[rf_ix], Bin)
    @variable(m, rr[rr_ix], Bin)


    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=0:1],

                # outflow
                sum(ff[key] for key in ff_ix_arr[c]
                        if (key[2] == g) & (key[4] == t) & (key[6] == rest)
                    ) +    

                sum(fr[key] for key in fr_ix_arr[c]
                            if (key[2] == g) & (key[3] == t) & (key[5] == rest)
                    ) 

        ==
                # inflow
                sum(ff[key] for key in ff_ix_arr[c]
                            if (key[3] == g) & (key[5] == t) & (key[6] == rest)
                    ) +

                sum(rf[key] for key in rf_ix_arr[c]
                            if (key[2] == g) & (key[4] == t) & (key[5] == rest)
                    ) 

                )   

    @constraint(m, rest_flow[c=1:C, t=1:T, rest=0:1], 

                # outflow
                sum(rf[key] for key in rf_ix_arr[c]
                            if (key[3] == t) & (key[5] == rest)
                    ) +

                sum(rr[key] for key in rr_ix_arr[c]
                            if (key[2] == t) & (key[4] == rest)
                    )

                ==       

                # inflow
                sum(fr[key] for key in fr_ix_arr[c]
                            if (key[4] == t) &  (key[5] == rest)
                    ) +
                sum(rr[key] for key in rr_ix_arr[c]
                            if (key[3] == t) & (key[5] == rest)
                    )
               )

    @constraint(m, start[c=1:C], 

        sum(ff[key] for key in from_start_ff if key[1] == c) + 
        sum(rf[key] for key in from_start_rf if key[1] == c) + 
        sum(fr[key] for key in from_start_fr if key[1] == c) + 
        sum(rr[key] for key in from_start_to_non_rested_rr if key[1] == c) + 
        sum(rr[key] for key in from_start_to_yes_rested_rr if key[1] == c)== 1
               )


    # linking constraints
    @constraint(m, linking[g=1:G, t=1:T],

         sum(ff[key] for key in ff_ix if (key[3] == g) & (key[5] == t)) + 
         sum(rf[key] for key in rf_ix if (key[2] == g) & (key[4] == t))
            >= supp_plans[plans_arr[g], g, t]

               );

    @objective(m, Min, 
        sum(ff[key] * (alpha + fire_dists[key[2], key[3]]) for key in ff_ix) +
        sum(fr[key] * (base_fire_dists[key[1], key[2]]) for key in fr_ix) + 
        sum(rf[key] * (alpha + base_fire_dists[key[1], key[2]]) for key in rf_ix)
    )

    optimize!(m)
    
    opt = false
    fires_fought = [(0)]
    route_costs = [1]
    if termination_status(m) == MOI.OPTIMAL
        opt = true
        fires_fought = vcat([(ix[1], ix[3], ix[5]) for ix in ff_ix if (value(ff[ix]) > 0.99)],
                        [(ix[1], ix[2], ix[4]) for ix in rf_ix if (value(rf[ix]) > 0.99)])
        
        route_costs = 
        [sum(value(ff[key]) * (alpha + fire_dists[key[2], key[3]]) for key in ff_ix_arr[crew]) +
        sum(value(fr[key]) * (base_fire_dists[key[1], key[2]]) for key in fr_ix_arr[crew]) + 
        sum(value(rf[key]) * (alpha + base_fire_dists[key[1], key[2]]) for key in rf_ix_arr[crew])
            for crew = 1:C]
    end
    opt, fires_fought, route_costs
end